In [ ]:
pip install opencv-python


In [ ]:
pip install torch torchvision facenet-pytorch pillow


In [ ]:
import os
from pathlib import Path
from PIL import Image
import torch
from facenet_pytorch import MTCNN

# Define your working directories and class names
root_dir = r"C:\Users\georg\OneDrive\Desktop\Capsule\Dataset 2"
output_dir = os.path.join(root_dir, "processed image")
classes = ["White", "Brown", "Black"]

# Ensure output folders are created
for cls in classes:
    Path(os.path.join(output_dir, cls)).mkdir(parents=True, exist_ok=True)

# Set up device and face detector
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

mtcnn = MTCNN(keep_all=False, image_size=160, margin=20, device=device)

def process_folder(class_name):
    in_dir = os.path.join(root_dir, class_name)
    out_dir = os.path.join(output_dir, class_name)
    for fname in os.listdir(in_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(in_dir, fname)
            try:
                img = Image.open(img_path).convert("RGB")
            except Exception as e:
                print(f"Failed to open {img_path}: {e}")
                continue
            # Detect and crop the largest face
            face_cropped = mtcnn(img)
            if face_cropped is None:
                print(f"No face detected in {fname}")
                continue
            # Ensure output is uint8 for PIL compatibility
            face_img = face_cropped.permute(1, 2, 0).byte().cpu().numpy()
            face_save = Image.fromarray(face_img)
            out_path = os.path.join(out_dir, fname)
            face_save.save(out_path)
            print(f"Processed {fname} -> {out_path}")

# Process each class folder
for cls in classes:
    process_folder(cls)


In [2]:
#new code 

In [4]:
pip install torch torchvision opencv-python mediapipe tqdm scikit-learn pillow


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

IMG_SIZE = 224  # Input size for DenseNet
SRC_PATH = r"C:\Users\georg\OneDrive\Desktop\Capsule\dataset 3"
SAVE_PATH = r"C:\Users\georg\OneDrive\Desktop\Capsule\images"

def extract_cheek_forehead(image):
    mp_face = mp.solutions.face_mesh
    with mp_face.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            return None
        h, w = image.shape[:2]
        landmarks = results.multi_face_landmarks[0].landmark
        pts = np.array([[int(l.x * w), int(l.y * h)] for l in landmarks])
        forehead = pts[10]
        left_cheek = pts[234]
        right_cheek = pts[454]
        x0 = max(0, min(forehead[0], left_cheek[0], right_cheek[0]) - 40)
        y0 = max(0, forehead[1] - 70)
        x1 = min(w, max(left_cheek[0], right_cheek[0]) + 40)
        y1 = min(h, max(left_cheek[1], right_cheek[1]) + 80)
        roi = image[y0:y1, x0:x1]
        return cv2.resize(roi, (IMG_SIZE, IMG_SIZE))

def process_dataset(src_folder, save_folder):
    os.makedirs(save_folder, exist_ok=True)
    for fname in tqdm(os.listdir(src_folder)):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        img_path = os.path.join(src_folder, fname)
        im = cv2.imread(img_path)
        if im is None:
            continue
        roi = extract_cheek_forehead(im)
        if roi is None:
            continue
        save_path = os.path.join(save_folder, fname)
        cv2.imwrite(save_path, roi)
    print("Processing complete.")

if __name__ == "__main__":
    process_dataset(SRC_PATH, SAVE_PATH)


100%|████████████████████████████████████████████████████████████████████████████████| 918/918 [09:06<00:00,  1.68it/s]

Processing complete.


In [12]:
import cv2
import numpy as np

# Path to your reference image
palette_path = r"C:\Users\georg\Downloads\skin-tones-palette-by-name-different-types-human-skin-flat-icon-set-vector.jpg"  # update accordingly

# Names and coordinates of patches (adjust x,y,w,h for your palette!)
skin_tones = [
    ("PORCELAIN",    (30,  65, 60, 40)),
    ("IVORY",        (130, 65, 60, 40)),
    ("WARM_IVORY",   (235, 65, 60, 40)),
    ("SAND",         (340, 65, 60, 40)),
    ("ROSE_BEIGE",   (445, 65, 60, 40)),
    ("NATURAL",      (30,  125, 60, 40)),
    ("BEIGE",        (130, 125, 60, 40)),
    ("SIENA",        (235, 125, 60, 40)),
    ("BAND",         (340, 125, 60, 40)),
    ("ALMOND",       (445, 125, 60, 40)),
    ("AMBER",        (30,  185, 60, 40)),
    ("UMBER",        (130, 185, 60, 40)),
    ("BRONZE",       (235, 185, 60, 40)),
    ("ESPRESSO",     (340, 185, 60, 40)),
    ("CHOCOLATE",    (445, 185, 60, 40)),
]

img = cv2.imread(palette_path)
skin_tone_rgbs = {}

for label, (x, y, w, h) in skin_tones:
    patch = img[y:y+h, x:x+w]
    avg_color = patch.mean(axis=(0,1))  # BGR format
    avg_color_rgb = avg_color[::-1]     # Convert to RGB
    skin_tone_rgbs[label] = tuple(map(int, avg_color_rgb))
    print(f"{label}: {skin_tone_rgbs[label]}")

# Optional: save to file (for later use)
import json
with open('skin_tone_rgb_values.json', 'w') as f:
    json.dump(skin_tone_rgbs, f, indent=2)


PORCELAIN: (255, 255, 255)
IVORY: (225, 225, 225)
WARM_IVORY: (212, 212, 212)
SAND: (213, 213, 213)
ROSE_BEIGE: (209, 209, 209)
NATURAL: (253, 247, 241)
BEIGE: (246, 228, 211)
SIENA: (238, 220, 200)
BAND: (242, 230, 198)
ALMOND: (226, 205, 183)
AMBER: (251, 247, 243)
UMBER: (244, 231, 216)
BRONZE: (245, 230, 212)
ESPRESSO: (234, 219, 208)
CHOCOLATE: (221, 210, 198)


In [14]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import shutil
import json

INPUT_FOLDER = r"C:\Users\georg\OneDrive\Desktop\Capsule\images"
OUTPUT_FOLDER = r"C:\Users\georg\OneDrive\Desktop\Capsule\classified images"

# Load the RGB dictionary just created
with open('skin_tone_rgb_values.json', 'r') as f:
    SKIN_TONE_COLORS = json.load(f)
for k in SKIN_TONE_COLORS:   # dict values could be lists, convert to tuple
    SKIN_TONE_COLORS[k] = tuple(SKIN_TONE_COLORS[k])

def avg_patch_color(image):
    pixels = image.reshape(-1, 3)
    avg = np.median(pixels, axis=0)
    return avg

def closest_skin_tone(color):
    min_dist = float('inf')
    label = None
    for name, ref_rgb in SKIN_TONE_COLORS.items():
        dist = np.linalg.norm(color - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            label = name
    return label

def classify_and_save_images():
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for fname in tqdm(files):
        fpath = os.path.join(INPUT_FOLDER, fname)
        img = cv2.imread(fpath)
        if img is None: continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        avg_color = avg_patch_color(img_rgb)
        label = closest_skin_tone(avg_color)
        target_dir = os.path.join(OUTPUT_FOLDER, label)
        os.makedirs(target_dir, exist_ok=True)
        shutil.copy(fpath, os.path.join(target_dir, fname))

if __name__ == "__main__":
    classify_and_save_images()
    print("Done: classified images saved by skin tone.")


100%|████████████████████████████████████████████████████████████████████████████████| 868/868 [00:17<00:00, 48.44it/s]

Done: classified images saved by skin tone.
